In [1]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

res = client.list_collections()

print("collections are: ", res)



collections are:  ['_emails_attachments']


In [3]:
import json

res = client.describe_collection(
    collection_name="_emails_attachments"
)

print(json.dumps(res, indent=4))


{
    "collection_name": "_emails_attachments",
    "auto_id": false,
    "num_shards": 1,
    "description": "",
    "fields": [
        {
            "field_id": 100,
            "name": "id",
            "description": "",
            "type": 21,
            "params": {
                "max_length": 65535
            },
            "is_primary": true
        },
        {
            "field_id": 101,
            "name": "embedding",
            "description": "",
            "type": 101,
            "params": {
                "dim": 1536
            }
        }
    ],
    "functions": [],
    "aliases": [],
    "collection_id": 456905014728701700,
    "consistency_level": 1,
    "properties": {},
    "num_partitions": 1,
    "enable_dynamic_field": true,
    "created_timestamp": 456926605253804039
}


In [4]:
from utils.config_loader import app_config
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

In [5]:
# 1. Connect to Milvus
milvus_store = MilvusVectorStore(
    uri=app_config.MILVUS_URL,
    dim=app_config.MILVUS_DIM,
    overwrite= False,
    collection_name="_emails_attachments"
)

2025-04-17 04:41:43,039 [DEBUG][_create_connection]: Created new connection using: d3cece27f54b4d3c878b440bc46f19de (async_milvus_client.py:600)


### Example 1: Using LlamaIndex to query the vector store

In [6]:
## Step 1: Prepare global settings

# Access appplication environment variables
model = app_config.LLAMAINDEX_MODEL
embed_model= app_config.LLAMAINDEX_EMBEDDING

# define global Setting that sets the LLM and ebemdding model.
# There is no need to pass in any llm or embedding model to the LlamaIndexTool query engine.
# The LlamaIndexTool will use the global Settings to get the llm and embedding model.
Settings.llm = OpenAI(temperature=0.1, model=model)
Settings.embed_model = OpenAIEmbedding(model=embed_model)

In [7]:
# Step 2: Load index from existing Milvus vector store
index = VectorStoreIndex.from_vector_store(
    vector_store=milvus_store
)

In [8]:
# Step 3. Create the query and chat engine using the LLM
query_engine = index.as_query_engine()  # no parameter needed, taken care off by the global Settings
chat_engine = index.as_chat_engine()


In [14]:

# Step 4A. Run your query
question = "what are the terms and conditions for PT_Interindo"
response = query_engine.query(question)
print(response)

The terms and conditions for PT Interindo include ensuring that all necessary documentation such as commercial invoices, packing lists, and permits are prepared for shipment. It is also important to comply with the key regulations for importing and exporting in Singapore by declaring goods via TradeNet® and ensuring no restricted or prohibited goods are included. Additionally, monitoring the vessel ETA and container release status through the PortNet platform is advised to ensure smooth progress of the shipment. For further specific guidelines, reaching out directly to PT Interindo is recommended.


In [16]:
question = "what is the vessel information  PT_Interindo request"
response = query_engine.query(question)
print(response)

The vessel information requested by PT Interindo includes the estimated time of arrival (ETA) in Singapore on September 10, 2024, and the ETA in Jakarta.
